In [ ]:
#final project for network class 

In [ ]:

#fake quantum environment for variational circuit optimization

import numpy as np
from gym import spaces
import gym

class quantumcircuitEnv(gym.Env):
    def __init__(self):
        super().__init__()
        self.n_params = 3
        self.action_space = spaces.Discrete(6)  # 6 actions
        self.observation_space = spaces.Box(
            low=-3.14, high=3.14, shape=(5,), dtype=np.float32
        )
        self.params = np.zeros(3)
        self.steps = 0
        self.target = 1.0
        
    def reset(self):
        self.params = np.random.uniform(-np.pi, np.pi, size=3)
        self.steps = 0
        return self._get_obs()
    
    def _quantum_circuit(self, params):  #fake quantum circuit, will be replaced with real one later
        rx, ry, rz = params
        expectation = np.cos(rx) * np.sin(ry) * np.cos(rz)
        return expectation
    
    def _get_obs(self):
        expectation = self._quantum_circuit(self.params)
        loss = (expectation - self.target) ** 2
        return np.array([*self.params, expectation, loss], dtype=np.float32)
    
    def step(self, action):
        param_idx = action // 2
        direction = 1 if action % 2 == 0 else -1
        
        self.params[param_idx] += direction * 0.1
        self.params = np.clip(self.params, -np.pi, np.pi)
        
        obs = self._get_obs()
        loss = obs[4]
        
        reward = -loss
        if loss < 0.01:
            reward += 50
        
        self.steps += 1
        done = self.steps >= 50 or loss < 0.01
        
        return obs, reward, done, {}

In [ ]:
#info for the gradient landscape, loss calculation, is sent to touchdesigner. 

#how do make 
#import libary 
# define model and data 
#create loss metric
#generate 2D landscape
#then plot the surface 

import loss_landscapes
import loss_landscapes.metrics

#metric for loss landscape
# Define loss function
    criterion = nn.CrossEntropyLoss()
    
    # Create metric
    metric = loss_landscapes.metrics.Loss(criterion, x, y)
    
    # Generate landscape
    loss_data = loss_landscapes.random_plane(
        model,
        metric,
        distance=1,
        steps=40,
        normalization='filter',
        deepcopy_model=True
    )
    
    #plot 
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    X = np.array([[j for j in range(40)] for i in range(40)])
    Y = np.array([[i for _ in range(40)] for i in range(40)])
    
    ax.plot_surface(X, Y, loss_data, cmap='viridis')
    
    ax.set_xlabel('Direction 1')
    ax.set_ylabel('Direction 2')
    ax.set_zlabel('Loss')
    
    plt.show() 

#go on claude ai and read where the sections of code i need to look at is.
#really read adnd understand the plot_surface section. 
#  should be done by monday 
#then i can do the MQTT connecting the loss landscape to touchdesigner


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1, 5)]            0         
                                                                 
 dense_12 (Dense)            (None, 1, 32)             192       
                                                                 
 flatten_3 (Flatten)         (None, 32)                0         
                                                                 
 dense_13 (Dense)            (None, 32)                1056      
                                                                 
 dense_14 (Dense)            (None, 6)                 198       
                                                                 
Total params: 1,446
Trainable params: 1,446
Non-trainable params: 0
_________________________________________________________________


In [21]:
#build reinforcement learning agent

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn
dqn = build_agent(model, actions=6)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])



In [24]:
# test and train agent in environment
dqn.fit(env, nb_steps=5000, visualize=False, verbose=1)
dqn.test(env, nb_episodes=10, visualize=False)

Training for 5000 steps ...
Interval 1 (0 steps performed)
 4998/10000 [=============>................] - ETA: 24s - reward: -0.5610done, took 24.128 seconds
Testing for 10 episodes ...
Episode 1: reward: -31.933, steps: 50
Episode 2: reward: 49.253, steps: 11
Episode 3: reward: -68.364, steps: 50
Episode 4: reward: -33.669, steps: 50
Episode 5: reward: 43.504, steps: 18
Episode 6: reward: -50.000, steps: 50
Episode 7: reward: -33.933, steps: 50
Episode 8: reward: -4.812, steps: 50
Episode 9: reward: 30.451, steps: 37
Episode 10: reward: 9.787, steps: 43
